Restrições hard:
Não podemos alocar uma turma numa sala com quantidade de assentos menor
que o número de alunos matriculados.
Não podemos alocar uma turma que necessita de acessibilidade numa sala que
não é acessível
Restrições soft
• Devemos minimizar a taxa de ocupação de salas: (número de alunos
– tamanho da sala)/tamanho da sala.
• Restrição de qualidade da sala: Uma turma de qualidade baixa pode
ser alocada numa sala de qualidade alta, mas o contrário não é
verdade.
• Restrição de salas no mesmo horário: penalizar uma distribuição
onde uma mesma turma esteja numa sala diferentes.
• Restrição de calouros na mesma sala: penalizar distribuições onde
turmas de calouros de um mesmo curso estejam em salas
diferentes.

In [237]:
#definitions

import csv
import math
import random
#leitura andre
df = csv.DictReader(open("cenario2-salas.csv", encoding='utf-8')) 
df1 = csv.DictReader(open("cenario2-turmas.csv", encoding='utf-8'))

#leitura paulo
#df = csv.DictReader(open(r"C:\Users\Paulo\Desktop\TrabalhoSAD\cenario1-salas.csv",encoding='utf-8'))
#df1 = csv.DictReader(open(r"C:\Users\Paulo\Desktop\TrabalhoSAD\cenario1-turmas.csv",encoding='utf-8'))
turmas = {}
salas = {}
horarios = {}
id_Salas = []
horario = {}
x = 0
pesoOcup = 1
pesoAcess = 1
pesoQuali = 1

In [238]:
#transforma o horario para um formato mais amigavel
def transHorario(horario):
    list = []
    turmaTemp = horario.split("-")
    for i in range(len(turmaTemp)):
        dia = int(turmaTemp[i][0])
        turno = turmaTemp[i][1]
        horarioIni = int(turmaTemp[i][2])

        if(turno == 'M'):
            horarioIni = 1 + horarioIni/2 
        if(turno == 'T'):
            horarioIni = 4 + horarioIni/2 

        elif(turno == 'N'):
            horarioIni = 7 + horarioIni/2
        
        if(int(turmaTemp[i][0]) == 2):
            dia = 'seg'
        elif(int(turmaTemp[i][0]) == 3):
            dia = 'ter'
        elif(int(turmaTemp[i][0]) == 4):
            dia = 'qua'
        elif(int(turmaTemp[i][0]) == 5):
            dia = 'qui'
        elif(int(turmaTemp[i][0]) == 6):
            dia = 'sex'
        else:
            dia = 'sab'
        list.append((dia,int(horarioIni)))
    return list

In [239]:
    
for row in df1:
    turmas[x] = {"disciplina": row['disciplina'],"prof": row['professor'], "horario": transHorario(row['dias_horario']),
                 "alunos": int(row['numero_alunos']), "curso": row['curso'], 
                 "periodo": int(row['período']), "acess": int(row["acessibilidade"]), "quali": int(row["qualidade"])}
    x+=1
for row in df:
    salas[row['id_sala']] = {"cad": int(row['numero_cadeiras']), "acess": int(row['acessivel']), "quali": int(row['qualidade'])}
    id_Salas.append(row['id_sala'])

for sala in id_Salas:
    horarios[sala] =  { "seg":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "ter":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "qua":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "qui":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "sex":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "sab":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 }}




KeyError: 'numero_cadeiras'

In [188]:

sorted_salas = sorted(salas.items(), key=lambda sala: (sala[1]['acess'],sala[1]['cad']), reverse=True)

sorted_turmas = sorted(turmas.items(), key=lambda turma: (turma[1]['acess'], turma[1]['alunos']), reverse=True)


In [189]:
def achaSalasDisponiveis(horarios, turma, salas):
    salasPossiveis = achaSalasPossiveis(horarios, turma, salas)
    h = turma['horario']
    #recebe a lista de horarios dessa turma
    salasDisponiveis = []
    #lista que recebe a solução
    for sala in salasPossiveis:
        
        for j in range(len(horarios)):
            contador = 0
            for k in range(len(h)):
                dia = h[k][0]
                #recebe o dia do horario k 
                horario = h[k][1] 
                #recebe a hora do horario k
                if(horarios[sala[0]][dia][horario] == 0): 
                    contador += 1 
                    #se o horario estiver disponivel é incrementado este contador
        if(contador == len(h)): 
            #se o contador chegar ao mesmo tamanho da lista de horario, significa que essa 
            # sala está livre em todos os horarios dessa turma, logo ela pode ser alocada.
            salasDisponiveis.append(sala) #adiciona a sala avaliada na lista resultado
    return salasDisponiveis

In [227]:
def achaSalasPossiveis(horarios, turma, salas):
    #busca as salas que ainda estão livres
    salasPossiveis = {}
    for sala in salas:
        # Verifica a acessibilidade e o tamanho da turma. Ex:
        # Acess = 0 retornar turmas com acess 0 e 1
        # Acess = 1 retornar turmas com acess 1
        if(sala[1]['cad'] >= turma['alunos'] and sala[1]['acess'] >= turma['acess']): 
            salasPossiveis[sala[0]] = {"cad": int(sala[1]['cad']), "acess": int(sala[1]['acess']), "quali": int(sala[1]['quali'])}
            
        salas = sorted(salasPossiveis.items(), key=lambda sala: (sala[1]['acess'],sala[1]['cad']))
        print(salas)
    return salas

In [228]:

def alocaTurmasIngenua(horarios, turmas, salas):
    
    salasDisponiveis = []
    
    for turma in turmas:
        
        h = turma[1]['horario']
        salasDisponiveis = achaSalasDisponiveis(horarios, turma[1], salas)
        
        if(salasDisponiveis == ()):
            print("deu conflito rs", turma[1]['alunos'])
        else:
            for i in range(len(h)):
                dia = h[i][0] 
                #recebe o dia do horario k 
                horario = h[i][1] 
                #recebe a hora do horario k
                sala = salasDisponiveis[0][0]
                horarios[sala][dia][horario] = turma[0] #coloca o id(lista não ordenada) da turma no horario.
                
    return horarios
            

In [229]:
#calcula a taxa de ocupação de uma sala em relação a uma turma
def taxaOcupacao(turma, sala):
    print(turma['alunos'], sala['cad'])
    print(turma['alunos']/sala['cad'])
    return turma['alunos']/sala['cad']

#retorna 1 caso a sala esteja em uma sala de qualidade adequada
#e retorna 0 caso contrario
def taxaQualidade(turma, sala):
    if sala['quali'] >= turma['quali']:
        return 1
    return 0

#retorna 1 caso a sala esteja em uma sala de acessibilidade adequada
#e retorna 0 caso contrario
def taxaAcessibilidade(turma, sala):
    if (sala['acess'] >= turma['acess']):
        return 1
    return 0

In [230]:
#analisa qualidade da solução atual
def analisaQualidade(solucao, turmas, salas):
    somatorio = []
    for i in range(len(solucao)):
        # loop que passa pelas salas
        id_sala = solucao[i][0]
        for dia,value in solucao[i][1].items():
            # Loop que passa pelos dias e seus valores
            # dia  = seg, ter ... | Value = 1: idTurma, 2: 0 ...
            for horario, id_turma in value.items():
                # Verifica se teve aula no dia.
                # Caso sim analisa as taxas de qualidade para essa turma
                if id_turma > 0:
                    
                    ocup = taxaOcupacao(turmas[id_turma], salas[id_sala])
                    quali = taxaQualidade(turmas[id_turma], salas[id_sala])
                    acess = taxaAcessibilidade(turmas[id_turma], salas[id_sala]) 
                    #print(ocup, quali, acess)
                    #print("______________________________________________________________")
                    obj = {"ocup": ocup,"acess": acess ,"quali": quali}
                    somatorio.append(obj)
    return somatorio

def qualidadeDaSolucao(solucao, turmas, salas):

    #calcula a qualiade da solução no geral
    #quanto mais as taxas se aproximem de 1,
    # melhor é a qualidade da solução
    erros = analisaQualidade(solucao, turmas, salas)
    sumOcup, sumQuali, sumAcess = (0,0,0)
    qualidadeFinal = 0 
    for erro in erros:
        sumOcup += erro['ocup']
        sumAcess += erro['acess']
        sumQuali += erro['quali']
    
    sumQuali = (sumQuali/len(erros))*pesoQuali 
    sumAcess = (sumAcess/len(erros))*pesoAcess  
    sumOcup  = (sumOcup/len(erros))*pesoOcup 

    qualidadeFinal = (sumQuali + sumAcess + sumOcup)/ (pesoQuali + pesoAcess + pesoOcup)
    print()
    print("erro total é:", qualidadeFinal) 
    return(qualidadeFinal)



In [231]:
def exibeSolucao(solucao, turmas, salas):
    # Escreve a Sala seguido do seu horario na semana
    # Somente os horarios com aulas
    # Avisa caso a sala não tenha aula no dia
    for i in range(len(solucao)):
        # loop que passa pelas salas
        print("Sala:", solucao[i][0])
        for key,value in solucao[i][1].items():
            # Loop que passa pelos dias e seus valores
            # Key = seg, ter ... | Value = 1: idTurma, 2: 0 ...
            teveAula = False
            print("{}:".format(key))
            for chave, valor in value.items():
                # Verifica se teve aula no dia.
                # Caso sim escreve o horario e a turma.
                if valor > 0:
                    teveAula = True
                    print("Horario:",chave, '-',turmas[valor]['disciplina'])
            if teveAula ==False:
                # Caso não escreve a sala e o dia que não teve.
                print("Não tem aula na sala {} para {}".format(solucao[i][0],key))
        if i != len(solucao)-1:
            print("-------------------------------------------------------------------")

In [232]:
def achaSucessor(turmas, salas):
    temp = {}
    salasDisponiveis = []
    for sala in id_Salas:
        temp[sala] =  { "seg":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "ter":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "qua":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "qui":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "sex":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "sab":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 }}
    
    for turma in turmas:    
        h = turma[1]['horario']
        salasDisponiveis = achaSalasDisponiveis(temp, turma[1], salas)
        
        if(salasDisponiveis == []):
            print("deu conflito rs", turma[1]['alunos'])
        else:
            for i in range(len(h)):
                dia = h[i][0] 
                #recebe o dia do horario k 
                horario = h[i][1] 
                #recebe a hora do horario k
                r = random.randrange(len(salasDisponiveis))
                sala = salasDisponiveis[r][0]
                temp[sala][dia][horario] = turma[0] #coloca o id(lista não ordenada) da turma no horario.
                
    return temp
            


In [233]:
def trocarTurma(horarios, salas, dias, turma):
    #original = avaliaTurmas(horarios, salas, dias, turmas);
    horarios[salas[0]][dias[0]][turma], horarios[salas[1]][dias[1]][turma] = horarios[salas[1]][dias[1]][turma], horarios[salas[0]][dias[0]][turma]
    #novo = avaliaTurmas(horarios, salas, dias, turmas)
    #melhor = avaliaTroca(original, novo)
    return horarios
#troca = trocaTurma(horarios, ['101','101'], ['seg','ter'], 2)

In [234]:
def simulatedAnnealing (solucao, turmas, salas, temp, maxIterations, alvo, alfa):
    sucessores = achaSalasPossiveis
    init = solucao 
    qualiInit = qualidadeDaSolucao(init, turmas, salas)
    i = 0

    while (temp > 1 ):
        while (i < maxIterations):
            sucessor = list(achaSucessor(sorted_turmas, sorted_salas).items())
            qualiSucessor = qualidadeDaSolucao(sucessor, turmas, salas)
            qualiInit = qualidadeDaSolucao(init, turmas, salas)
            if(qualiSucessor > qualiInit):
                init = sucessor
                qualiInit = qualiSucessor
            else:
                erro = -qualiInit - qualiSucessor
                probabilidade = math.exp(erro/temp)
                print("probabilidade", probabilidade)
                if(random.random() < probabilidade):
                    init = sucessor
                    qualiInit = qualiSucessor
            if(qualiInit > alvo):
                return init 
            i += 1
        temp *= alfa 
        i = 0

    qualiSolucao = qualidadeDaSolucao(solucao, turmas, salas)
    if(qualiInit > qualiSolucao ):
        return init
    else: 
        return solucao
        


In [236]:
solucaoIngenua =  list(alocaTurmasIngenua(horarios, sorted_turmas, sorted_salas).items())
qualidadeDaSolucao(solucaoIngenua, turmas, salas)
#exibeSolucao(solucaoIngenua, turmas, salas)

#solucaoAleatoria = list(achaSucessor(sorted_turmas, sorted_salas).items())
#qualidadeDaSolucao(solucaoAleatoria, turmas, salas)
#exibeSolucao(solucaoAleatoria, turmas, salas)

print()
#x = simulatedAnnealing (solucaoIngenua, turmas, salas, 10000.0 , 50, 0.54, 0.45)
#exibeSolucao(x, turmas, salas)
#qualidadeDaSolucao(x, turmas, salas)

[('101', {'cad': 100, 'acess': 1, 'quali': 1})]
[('101', {'cad': 100, 'acess': 1, 'quali': 1}), ('201', {'cad': 100, 'acess': 1, 'quali': 2})]
[('101', {'cad': 100, 'acess': 1, 'quali': 1}), ('201', {'cad': 100, 'acess': 1, 'quali': 2}), ('301', {'cad': 100, 'acess': 1, 'quali': 2})]
[('101', {'cad': 100, 'acess': 1, 'quali': 1}), ('201', {'cad': 100, 'acess': 1, 'quali': 2}), ('301', {'cad': 100, 'acess': 1, 'quali': 2})]
[('101', {'cad': 100, 'acess': 1, 'quali': 1}), ('201', {'cad': 100, 'acess': 1, 'quali': 2}), ('301', {'cad': 100, 'acess': 1, 'quali': 2})]
[('101', {'cad': 100, 'acess': 1, 'quali': 1}), ('201', {'cad': 100, 'acess': 1, 'quali': 2}), ('301', {'cad': 100, 'acess': 1, 'quali': 2})]
[('101', {'cad': 100, 'acess': 1, 'quali': 1}), ('201', {'cad': 100, 'acess': 1, 'quali': 2}), ('301', {'cad': 100, 'acess': 1, 'quali': 2})]
[('101', {'cad': 100, 'acess': 1, 'quali': 1}), ('201', {'cad': 100, 'acess': 1, 'quali': 2}), ('301', {'cad': 100, 'acess': 1, 'quali': 2})]
[('10

>>> 0.5512510667349377 - 0.5096996074415429
0.04155145929339488




In [ ]:
0.5262962962962963